### Spotify API

In [1]:
import requests
import json
import logging
import base64
import pymysql
import csv

### Connecting to Amazon RDS

In [2]:
## mysql -h testdb-spotify.c1ljsqo7birx.us-west-2.rds.amazonaws.com -P 3306 -D production -u admin -p
host="testdb-spotify.c1ljsqo7birx.us-west-2.rds.amazonaws.com"
port=3306
username="admin"
password="Wlqtnsdl11061310*"
database="production"

connection=pymysql.connect(host=host, user=username, password=password, database=database, port=port, use_unicode=True, charset="utf8")
cursor=connection.cursor()

cursor.execute("SHOW TABLES")
print(cursor.fetchall()) 

(('artist_genres',), ('artists',), ('people',))


### Funtion to add rows in a table

In [3]:
# query = """
#INSERT INTO artists (id, name, follwers, popularity, url, image_url)
#VALUES ('{}', '{}', '{}', '{}', '{}', '{}')
#ON DUPILICATE KEY UPDATE id='{}', name='{}', follwers='{}', popularity='{}', url='{}', image_url='{}'
#""".format(artist['id'], artist['name'], artist['followers'], artist['popularity'], artist['url'], artist['image_url'],
#           artist['id'], artist['name'], artist['followers'], artist['popularity'], artist['url'], artist['image_url'])


def insert_row(cursor, data, table_name):
    
    column_name = ', '.join(data.keys())
    print('columns:', column_name)
    
    placeholders = ', '.join(['%s'] * len(data)) #len(data) = the number of columns
    #%s, %s, %s, %s, %s, %s

    key_placeholders = ', '.join(['{}=%s'.format(k) for k in data.keys()])
    #id=%s, name=%s, followers=%s, popularity=%s, url=%s, image_url=%s
    
    query = "INSERT INTO %s (%s) VALUES (%s) ON DUPLICATE KEY UPDATE %s" % (table_name, column_name, placeholders, key_placeholders)
    cursor.execute(query, list(data.values())*2)

### Access to Spotify API

In [4]:
client_id="b03942aa01eb477cacb26e9bf8d3da58"
response_type="code"
redirect_uri="http://localhost:8888"
#scope="user-read-private%20user-read-email"
endpoint1="https://accounts.spotify.com/authorize?client_id={}&response_type={}&redirect_uri={}".format(client_id, response_type, redirect_uri)
#endpoint1="https://accounts.spotify.com/authorize?client_id={}&response_type={}&redirect_uri={}&scope={}".format(client_id, response_type, redirect_uri, scope)
res1=requests.get(endpoint1)

print(res1.status_code)
print(res1.text)
print(endpoint1)

200

<!DOCTYPE html>
<html id="app" lang="en" dir="ltr" ng-csp ng-strict-di>
<head>
  <meta charset="utf-8">
  <title ng-bind="(&#39;loginTitle&#39; | localize) + &#39; - Spotify&#39;">Spotify</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
  <base href="/">
  <link rel="icon" href="https://accounts.scdn.co/sso/images/favicon.ace4d8543bbb017893402a1e9d1ac1fa.ico">
  <link href="https://accounts.scdn.co/sso/css/index.e55120e5a065760b808d.css" media="screen" rel="stylesheet">

  <script defer src="https://accounts.scdn.co/sso/js/index.e55120e5a065760b808d.js" sp-bootstrap></script>
  <meta ng-non-bindable sp-bootstrap-data='{"phoneFeatureEnabled":false,"previewEnabled":false,"user":false,"tpaState":"AQCe7dCLfcBDQgZJx3qzecJtt+dn8lBjDTPuPsmiXPSYEG6tDpgUx4skxqxpChMs7Acb8C9u7Eefl4uHoGUpBgE6dUCR4uT8ccOzxLKCQx4HJ8kW3bgWHfHjHiXZGhU6loF5VZCYL54G2XENVl+Tli099+OQtLXEQAVzxACBAUnVn37EZRsVzjjm1w7fiLq2/lxn2OJ0MNhuJEENDsCZ+zVWTjOnMFt

In [6]:
data={
    "grant_type":"authorization_code",
    "code":"AQBN0RkDfSZM04iXMlok_vs8ub7falpPxL_tS16EqfedPI_YWV89001dqVbdG4WoQv5yEq_8APV8C3YCOS-fYirDETBpyXFz7guVaXn_8B0NBBVGiF1PuLIQOkL51ysEBiJ5rgIC6tuM3IdCQ4AvlNiErChVRjtw4g",
    "redirect_uri":"http://localhost:8888"
}


client_id="b03942aa01eb477cacb26e9bf8d3da58"
client_secret="fd61807827344135a98ec16305e34bb2"
encoded=base64.b64encode('{}:{}'.format(client_id, client_secret).encode('utf-8')).decode('ascii')
headers={"Authorization":"Basic {}".format(encoded)}

endpoint2="https://accounts.spotify.com/api/token"

res2=requests.post(endpoint2, data=data, headers=headers)

print(res2.status_code)
print(res2.text) #type=string

200
{"access_token":"BQAJcI_JcoKp8mwCayyRDSuK9ANcPPuoSf5KrKiQJqEAldBbreRqweUE4KuTRX1cF81Z9HeoFyAZVMwnScBlgialO-2wYr8kbzAM4XFmNhAjoChr0z97ByubPHTS7aWr8AvcPcdMNO6gGZ3_YPjWTiGkAultrf2Qb5770A","token_type":"Bearer","expires_in":3600,"refresh_token":"AQA4bIeYFi_aPB1vq0p1eosAIm2CJEPBnXgQq_7J-trKOWsSsNFO2C92zLOWcQPSQVEUMofasN0tdPpZYYs63PavuIzzYgmN-2S5KsZ8IeYMS5MqjjaReJBBNAXHW7Xm5gs"}


In [8]:
print(res2.json())
json.loads(res2.text) #type=dict

access_token=json.loads(res2.text)['access_token']
headers2={"Authorization":"Bearer {}".format(access_token)}

{'access_token': 'BQAJcI_JcoKp8mwCayyRDSuK9ANcPPuoSf5KrKiQJqEAldBbreRqweUE4KuTRX1cF81Z9HeoFyAZVMwnScBlgialO-2wYr8kbzAM4XFmNhAjoChr0z97ByubPHTS7aWr8AvcPcdMNO6gGZ3_YPjWTiGkAultrf2Qb5770A', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQA4bIeYFi_aPB1vq0p1eosAIm2CJEPBnXgQq_7J-trKOWsSsNFO2C92zLOWcQPSQVEUMofasN0tdPpZYYs63PavuIzzYgmN-2S5KsZ8IeYMS5MqjjaReJBBNAXHW7Xm5gs'}


In [9]:
params={
    "q":"Dirty Honey",
    "type":"artist",
    "limit":"1"
}

res3=requests.get("https://api.spotify.com/v1/search", params=params, headers=headers2)
print(res3.status_code)
#print(res3.text)
#print(res3.json())

raw = json.loads(res3.text)
print(raw)
print(raw['artists'].keys())
print(raw['artists']['items'][0].keys())

artist_raw = raw['artists']['items'][0]

artist={}

if artist_raw['name'] == params['q']:
    artist.update({
        'id':artist_raw['id'],
        'name':artist_raw['name'],
        'followers':artist_raw['followers']['total'],
        'popularity':artist_raw['popularity'],
        'url':artist_raw['external_urls']['spotify'],
        'image_url':artist_raw['images'][1]['url']})

200
{'artists': {'href': 'https://api.spotify.com/v1/search?query=Dirty+Honey&type=artist&offset=0&limit=1', 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0XBRd3N11rCbh0s8qHf41m'}, 'followers': {'href': None, 'total': 77807}, 'genres': ['modern blues rock', 'modern hard rock'], 'href': 'https://api.spotify.com/v1/artists/0XBRd3N11rCbh0s8qHf41m', 'id': '0XBRd3N11rCbh0s8qHf41m', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebfa472bf2bffc5d890d160af6', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174fa472bf2bffc5d890d160af6', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178fa472bf2bffc5d890d160af6', 'width': 160}], 'name': 'Dirty Honey', 'popularity': 51, 'type': 'artist', 'uri': 'spotify:artist:0XBRd3N11rCbh0s8qHf41m'}], 'limit': 1, 'next': 'https://api.spotify.com/v1/search?query=Dirty+Honey&type=artist&offset=1&limit=1', 'offset': 0, 'previous': None, 'total': 3}}
dict_k

In [10]:
print(artist)
insert_row(cursor, artist, 'artists')
connection.commit()

{'id': '0XBRd3N11rCbh0s8qHf41m', 'name': 'Dirty Honey', 'followers': 77807, 'popularity': 51, 'url': 'https://open.spotify.com/artist/0XBRd3N11rCbh0s8qHf41m', 'image_url': 'https://i.scdn.co/image/ab67616100005174fa472bf2bffc5d890d160af6'}
columns: id, name, followers, popularity, url, image_url


### Inserting multiple rows of artists

In [57]:
artist_list = []

with open('artist_list.csv') as f:
    artist_list_raw = csv.reader(f)
    for row in artist_list_raw:
        artist_list.append(row[0])


# Get 1st~50th artists out of 500

for a in artist_list[0:49]:
    params2 = {
        'q':a,
        'type':'artist',
        'limit':'1'
    }
    
    r=requests.get("https://api.spotify.com/v1/search", params=params2, headers=headers2)
    raw2 = json.loads(r.text)
    
    artist2={}
    
    try:
        if raw2['artists']['items'][0]['name'] == params2['q']:
            artist2.update(
                 {'id': raw2['artists']['items'][0]['id'],
                  'name': raw2['artists']['items'][0]['name'],
                  'followers': raw2['artists']['items'][0]['followers']['total'],
                  'popularity': raw2['artists']['items'][0]['popularity'],
                  'url': raw2['artists']['items'][0]['external_urls']['spotify'],
                  'image_url': raw2['artists']['items'][0]['images'][1]['url']          
                 }
             )
                         
        
        insert_row(cursor, artist2, 'artists')
    
    
    except:
        logging.error(" No ITEMS FROM SEARCH API ")
        continue
                
                
connection.commit()        


ERROR:root: No ITEMS FROM SEARCH API 


columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url


ERROR:root: No ITEMS FROM SEARCH API 


columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url


ERROR:root: No ITEMS FROM SEARCH API 


...

columns: 
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url


ERROR:root: No ITEMS FROM SEARCH API 


columns: 
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url


ERROR:root: No ITEMS FROM SEARCH API 


columns: 
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url
columns: id, name, followers, popularity, url, image_url


### Inserting multiple artists in batch

In [15]:
cursor.execute("SELECT id FROM artists")
#cursor.fetchall()
cursor.fetchmany(15)

(('00FQb4jTyendYWaN8pK0wa',),
 ('02uYdhMhCgdB49hZlYRm9o',),
 ('03r4iKL2g2442PT9n2UKsx',),
 ('04gDigrS5kc9YWfZHwBETP',),
 ('05E3NBxNMdnrPtxF9oraJm',),
 ('06nevPmNVfWUXyZkccahL8',),
 ('06nsZ3qSOYZ2hPVIMcr1IN',),
 ('08avsqaGIlK2x3i2Cu7rKH',),
 ('09C0xjtosNAIXP36wTnWxd',),
 ('0BvkDsjIUla7X0k6CSWh1I',),
 ('0bvRYuXRvd14RYEE7c0PRW',),
 ('0C0XlULifJtAgn6ZNCW2eu',),
 ('0ECwFtbIWEVNwjlrfc6xoL',),
 ('0F3Aew9DSd6fb6192K1K0Y',),
 ('0FI0kxP0BWurTz8cB8BBug',))

In [34]:
artist_list2=[]
cursor.execute("SELECT id FROM artists")

for (id, ) in cursor.fetchall():
    artist_list2.append(id)

artist_batch = [artist_list2[i:i+50] for i in range(0, len(artist_list2), 50)]
artist_genres = []

for batch in artist_batch:
    ids = ','.join(batch)
    url = "https://api.spotify.com/v1/artists/?ids={}".format(ids)
    
    r2 = requests.get(url, headers=headers2)
    raw3 = json.loads(r2.text)
    
    for artist in raw3['artists']:
        for genre in artist['genres']:                     
            artist_genres.append(
            {
                'artist_id':artist['id'],
                'genre':genre
            
            }
            )
            
            
            #print(artist['genres']) ----> ['art pop', 'pop']
            #print(genre) --------------->'art pop'
            
            
print(artist_genres[0:15]) #For one artist id, there are several genres

[{'artist_id': '00FQb4jTyendYWaN8pK0wa', 'genre': 'art pop'}, {'artist_id': '00FQb4jTyendYWaN8pK0wa', 'genre': 'pop'}, {'artist_id': '02uYdhMhCgdB49hZlYRm9o', 'genre': 'alternative rock'}, {'artist_id': '02uYdhMhCgdB49hZlYRm9o', 'genre': 'blues rock'}, {'artist_id': '02uYdhMhCgdB49hZlYRm9o', 'genre': 'garage rock'}, {'artist_id': '02uYdhMhCgdB49hZlYRm9o', 'genre': 'garage rock revival'}, {'artist_id': '02uYdhMhCgdB49hZlYRm9o', 'genre': 'modern blues rock'}, {'artist_id': '02uYdhMhCgdB49hZlYRm9o', 'genre': 'modern rock'}, {'artist_id': '02uYdhMhCgdB49hZlYRm9o', 'genre': 'palm desert scene'}, {'artist_id': '02uYdhMhCgdB49hZlYRm9o', 'genre': 'rock'}, {'artist_id': '03r4iKL2g2442PT9n2UKsx', 'genre': 'alternative rock'}, {'artist_id': '03r4iKL2g2442PT9n2UKsx', 'genre': 'east coast hip hop'}, {'artist_id': '03r4iKL2g2442PT9n2UKsx', 'genre': 'golden age hip hop'}, {'artist_id': '03r4iKL2g2442PT9n2UKsx', 'genre': 'hip hop'}, {'artist_id': '03r4iKL2g2442PT9n2UKsx', 'genre': 'old school hip hop'

In [32]:
for data in artist_genres:
    insert_row(cursor, data, 'artist_genres')
    
connection.commit()

columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: art

columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: art

columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: art

columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: art

columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: artist_id, genre
columns: art